In [1]:
import pandas as pd
import pandas_datareader.data as web

/Users/markhendricks/Projects/finm-quant-2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import pandas_datareader as pdreader
pdreader.famafrench.get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
startdate = '19800101'
enddate = '20240531'

In [4]:
info = pd.DataFrame(columns=['Name', 'Unit', 'Construction', 'Description'])
info.loc['MKT'] = ['Market', 'Excess Return', 'Market-cap-weighted', 'US Equities']
info.loc['SMB'] = ['Size', 'Excess Return', 'Small Minus Big', 'Long small stocks and short big stocks']
info.loc['HML'] = ['Value', 'Excess Return', 'High Minus Low', 'Long value (high book-to-market) stocks and short growth (low book-to-market) stocks']
info.loc['RMW'] = ['Profitability','Excess Return', 'Robust Minus Weak', 'Long profitability (income statement) and short low profitability']
info.loc['CMA'] = ['Investment','Excess Return', 'Conservative Minus Agressive', 'Long stocks with low (conservative) investment (balance sheet) and short stocks with high (agressive) investment.']
info.loc['UMD'] = ['Momentum','Excess Return','Up Minus Down', 'Long stocks that have high recent returns and short stocks with low recent returns']
info.loc['RF'] = ['Risk-free rate', 'Total Return', 'Tbills', '']
info

,Name,Unit,Construction,Description
MKT,Market,Excess Return,Market-cap-weighted,US Equities
SMB,Size,Excess Return,Small Minus Big,Long small stocks and short big stocks
HML,Value,Excess Return,High Minus Low,Long value (high book-to-market) stocks and sh...
RMW,Profitability,Excess Return,Robust Minus Weak,Long profitability (income statement) and shor...
CMA,Investment,Excess Return,Conservative Minus Agressive,Long stocks with low (conservative) investment...
UMD,Momentum,Excess Return,Up Minus Down,Long stocks that have high recent returns and ...
RF,Risk-free rate,Total Return,Tbills,


In [5]:
rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
facs = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
facs = facs.to_timestamp().resample('M').last()
facs.rename(columns={'Mkt-RF':'MKT'},inplace=True)

rawdata = web.DataReader('F-F_Momentum_Factor', data_source='famafrench',start=startdate,end=enddate)
umd = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
umd = umd.to_timestamp().resample('M').last()
umd.columns = ['UMD']

rf = facs[['RF']]
facs = facs.join(umd).drop(columns=['RF'])
facs

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/3639308500.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/3639308500.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/3639308500.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  facs = facs.to_timestam

,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0551,0.0183,0.0175,-0.0170,0.0164,0.0755
1980-02-29,-0.0122,-0.0157,0.0061,0.0004,0.0268,0.0788
1980-03-31,-0.1290,-0.0693,-0.0101,0.0146,-0.0119,-0.0955
1980-04-30,0.0397,0.0105,0.0106,-0.0210,0.0029,-0.0043
1980-05-31,0.0526,0.0211,0.0038,0.0034,-0.0031,-0.0112
...,...,...,...,...,...,...
2023-12-31,0.0487,0.0732,0.0493,-0.0307,0.0132,-0.0551
2024-01-31,0.0070,-0.0574,-0.0238,0.0069,-0.0096,0.0511
2024-02-29,0.0506,-0.0078,-0.0349,-0.0199,-0.0214,0.0499


In [6]:
rawdata = web.DataReader('49_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
rets = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
rets = rets.to_timestamp().resample('M').last()
retsx = rets.subtract(rf['RF'],axis=0)
retsx

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/1804829268.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('49_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/1804829268.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('49_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_59849/1804829268.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read y

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0076,0.0285,0.0084,0.1009,-0.0143,0.1002,0.0362,0.0323,0.0048,0.0059,...,0.0158,0.0875,0.0465,-0.0126,0.0430,-0.0283,0.0258,0.0768,0.0308,0.0669
1980-02-29,0.0105,-0.0608,-0.0966,-0.0322,-0.0569,-0.0323,-0.0521,-0.0800,-0.0555,-0.0167,...,-0.0079,-0.0541,-0.0346,-0.0639,-0.0652,-0.0854,-0.0959,-0.0347,-0.0282,-0.0274
1980-03-31,-0.2224,-0.1119,-0.0167,-0.1469,-0.0193,-0.1271,-0.0826,-0.1237,-0.0566,-0.0668,...,-0.0819,-0.1509,-0.1098,-0.0906,-0.1449,-0.0560,-0.0880,-0.2451,-0.1254,-0.1726
1980-04-30,0.0449,0.0766,0.0232,0.0321,0.0830,-0.0529,0.0783,0.0153,0.0304,0.0115,...,0.0420,-0.0103,-0.0312,0.0353,0.0542,0.0728,0.0530,0.0977,0.0447,0.0769
1980-05-31,0.0632,0.0793,0.0457,0.0863,0.0815,0.0509,0.0324,0.0886,0.0560,0.0098,...,0.0564,0.1063,0.1142,0.0877,0.1134,0.0578,0.0557,0.0915,0.0844,0.0685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31,0.0629,0.0390,0.0183,0.0012,0.0048,0.0972,0.0355,0.0836,-0.0011,0.0419,...,0.0313,0.0687,0.0420,0.0595,0.0463,0.0925,-0.0101,0.1702,0.0968,0.0103
2024-01-31,-0.0500,-0.0201,-0.0119,-0.0129,-0.0298,-0.0610,0.0742,-0.0088,0.0330,-0.0563,...,-0.0167,-0.0203,0.0247,0.0092,0.0015,0.0109,0.0125,-0.0555,-0.0259,0.0408
2024-02-29,0.1574,-0.0134,0.0390,-0.0032,-0.0042,0.0328,0.0602,0.0107,0.0258,0.0556,...,0.0521,0.0807,0.0530,0.1040,0.0504,0.0362,0.0160,0.0425,0.0438,0.0667


In [7]:
with pd.ExcelWriter('../data/factor_pricing_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    facs.to_excel(writer, sheet_name='factors (excess returns)')
    retsx.to_excel(writer, sheet_name='portfolios (excess returns)')
    rf.to_excel(writer, sheet_name='risk-free rate')